# Jonathan Halverson
# Wednesday, December 6, 2017
# RNN applied to MNIST

While RNN's are usually used to predict the future (since they have memory in a sense), here we use one to do image classification.

In [1]:
import tensorflow as tf

In [2]:
tf.reset_default_graph()

In [3]:
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

In [4]:
learning_rate = 0.001

In [5]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])
L = tf.placeholder(dtype=tf.int32, shape=(None))
training = tf.placeholder_with_default(False, (None))

In [6]:
my_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
#my_cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, state_is_tuple=False)
outputs, states = tf.nn.dynamic_rnn(my_cell, X, dtype=tf.float32, sequence_length=L)

In [7]:
# use only the final state of the cell
fc_drop = tf.layers.dropout(states, rate=0.0, training=training)
logits = tf.layers.dense(fc_drop, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)

In [8]:
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [9]:
init = tf.global_variables_initializer()

In [10]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('/tmp/data/')
X_test = mnist.test.images.reshape(-1, n_steps, n_inputs)
y_test = mnist.test.labels

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [11]:
n_epochs = 30
batch_size = 150

In [12]:
L_cut = 7
L_batch = batch_size * [L_cut]
L_test = y_test.size * [L_cut]

In [13]:
with tf.Session() as sess:
     init.run()
     for epoch in range(n_epochs):
          for iteration in range(mnist.train.num_examples // batch_size):
               X_batch, y_batch = mnist.train.next_batch(batch_size)
               X_batch = X_batch.reshape((-1, n_steps, n_inputs))
               sess.run(training_op, feed_dict={X:X_batch, y:y_batch, L:L_batch, training:True})
          acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch, L:L_batch})
          acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test, L:L_test})
          lss_test = loss.eval(feed_dict={X:X_test, y:y_test, L:L_test})

          print(epoch, 'Train accuracy=', acc_train, "Test accuracy=", acc_test, "Test loss=", lss_test)

(0, 'Train accuracy=', 0.5933333, 'Test accuracy=', 0.54839998, 'Test loss=', 1.3362124)
(1, 'Train accuracy=', 0.49333334, 'Test accuracy=', 0.5589, 'Test loss=', 1.2733843)
(2, 'Train accuracy=', 0.55333334, 'Test accuracy=', 0.5729, 'Test loss=', 1.2261957)
(3, 'Train accuracy=', 0.58666664, 'Test accuracy=', 0.57980001, 'Test loss=', 1.2011247)
(4, 'Train accuracy=', 0.58666664, 'Test accuracy=', 0.58880001, 'Test loss=', 1.1844572)
(5, 'Train accuracy=', 0.67333335, 'Test accuracy=', 0.60119998, 'Test loss=', 1.1499432)
(6, 'Train accuracy=', 0.57999998, 'Test accuracy=', 0.60180002, 'Test loss=', 1.1403829)
(7, 'Train accuracy=', 0.63333333, 'Test accuracy=', 0.60570002, 'Test loss=', 1.1352493)
(8, 'Train accuracy=', 0.62, 'Test accuracy=', 0.60530001, 'Test loss=', 1.1226361)
(9, 'Train accuracy=', 0.5933333, 'Test accuracy=', 0.61360002, 'Test loss=', 1.1148288)
(10, 'Train accuracy=', 0.62666667, 'Test accuracy=', 0.61619997, 'Test loss=', 1.1091247)
(11, 'Train accuracy=', 0

|n_neurons|dropout_rate|seq_length|peak accuracy|
|----|----|----|----|
|150|0.00|28|97.2%|
|150|0.25|28|97.2%|
|150|0.50|28|97.0%|
|150|0.75|28|96.8%|
|150|0.99|28|74.2%|
|5|0.50|28|58.0%|
|5|0.00|28|52.0%|
|25|0.50|28|93.9%|
|25|0.00|28|94.6%|
|300|0.50|28|97.6%|
|150|0.50|14|93.3%|
|150|0.50|7|59.5%|
|150|0.00|7|63.3%|
|150|0.50|3|16.5%|

In [14]:
logits

<tf.Tensor 'dense/BiasAdd:0' shape=(?, 10) dtype=float32>

In [15]:
y

<tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=int32>

#### How does a simple linear model compare?

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier

X_train = mnist.train.images
y_train = mnist.train.labels

std_sc = StandardScaler()
X_train_std = std_sc.fit_transform(X_train)
X_test_std = std_sc.transform(X_test.reshape(-1, n_steps * n_inputs))

SGDClassifier(tol=1e-3, alpha=0.01, max_iter=1500).fit(X_train_std, y_train).score(X_test_std, y_test)

SGDClassifier(tol=1e-3, alpha=0.0001, max_iter=1500).fit(X_train_std, y_train).score(X_test_std, y_test)

SGDClassifier(tol=1e-3, alpha=0.00001, max_iter=1500).fit(X_train_std, y_train).score(X_test_std, y_test)

#### And a nonlinear model

from sklearn.ensemble import RandomForestClassifier

RandomForestClassifier(n_estimators=50).fit(X_train_std, y_train).score(X_test_std, y_test)

We see that the RNN outperforms RF. A lot more could be done to optimize the RNN as well as preprocess the data.